# Introducción

**Objetivo:**

Realizar el proceso de entendimiento de datos de WWImporters a través de perfilamiento general, análisis de reglas de negocio, estadísticos y análisis de calidad de datos. Este análisis ayudará a entender el comportamiento de los clientes o vendedores a partir de órdenes de venta con el fin de monitorear el logro de los objetivos estratégicos de WWImporters.

**Fuente de Datos:**

Base de datos WWImportersTransactional.


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [ ]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

# Carga de Datos

En esta sección, se cargan los datos necesarios para el análisis. Los datos se encuentran en archivos CSV y se importan a DataFrames de pandas.


In [30]:
import pandas as pd
import os

dataframes = []
csv_folder = './csv/'

for i in range(1, 25):
    file_name = os.path.join(csv_folder, f'{i}.csv')
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame()  # Crear un DataFrame vacío si el archivo no existe
    dataframes.append(df)

print(f'Se han importado {len(dataframes)} DataFrames')


Se han importado 24 DataFrames


# Perfilamiento General

Realizamos un análisis exploratorio inicial de las tablas importadas. A continuación, mostramos la estructura y primeras filas de cada DataFrame.


### Consulta SQL:
```sql
SELECT * FROM WWImportersTransactional.movimientosCopia LIMIT 10;


In [33]:
dataframes[0]

,TransaccionProductoID,ProductoID,TipoTransaccionID,ClienteID,InvoiceID,ProveedorID,OrdenDeCompraID,FechaTransaccion,Cantidad
0,118903,217,10,476,24904,NaN,NaN,"Apr 25,2014",-40
1,286890,135,10,33,60117,NaN,NaN,"Dec 10,2015",-7
2,285233,111,10,180,59768,NaN,NaN,"Dec 04,2015",-2
3,290145,213,10,33,60795,NaN,NaN,"Dec 23,2015",-3
4,247492,90,10,55,51851,NaN,NaN,"Jul 27,2015",-24
5,157714,212,10,146,33043,NaN,NaN,"Sep 15,2014",-20
6,250024,218,10,585,52377,NaN,NaN,"Aug 04,2015",-60
7,200801,120,10,30,42049,NaN,NaN,"Feb 23,2015",-3
8,221525,129,10,596,46388,NaN,NaN,"May 01,2015",-7
9,295331,197,10,1037,61886,NaN,NaN,"Jan 08,2016",-216


Esta consulta selecciona las primeras 10 filas de la tabla movimientosCopia. Es útil para obtener una visión preliminar del contenido y la estructura de los datos, ayudando a identificar las columnas disponibles y algunos ejemplos de los registros.

### Consulta SQL:
```sql
SELECT COUNT(*) AS total_registros FROM WWImportersTransactional.movimientosCopia;


In [34]:
dataframes[1]

,total_registros
0,204292


Esta consulta cuenta el número total de registros en la tabla movimientosCopia. Conocer el volumen total de datos es esencial para dimensionar el alcance del análisis y la carga de datos que se manejará.

### Consulta SQL:
```sql
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'movimientosCopia' AND TABLE_SCHEMA = 'WWImportersTransactional';



In [35]:
dataframes[2]

,COLUMN_NAME,DATA_TYPE
0,Cantidad,double
1,ClienteID,double
2,FechaTransaccion,text
3,InvoiceID,double
4,OrdenDeCompraID,text
5,ProductoID,int
6,ProveedorID,text
7,TipoTransaccionID,int
8,TransaccionProductoID,int


Esta consulta obtiene los nombres y tipos de datos de las columnas de la tabla movimientosCopia. Es fundamental para entender cómo están definidos los datos y qué tipo de información se maneja en cada columna. La estructura de la tabla es la siguiente:

- Cantidad (double): Representa la cantidad de productos movidos.
- ClienteID (double): Identificador del cliente involucrado en la transacción.
- FechaTransaccion (text): Fecha y hora en que se realizó la transacción.
- InvoiceID (double): Identificador de la factura asociada.
- OrdenDeCompraID (text): Identificador de la orden de compra.
- ProductoID (int): Identificador del producto movido.
- ProveedorID (text): Identificador del proveedor involucrado en la transacción.
- TipoTransaccionID (int): Identificador del tipo de transacción.
- TransaccionProductoID (int): Identificador único de la transacción del producto.

### Consulta SQL:
```sql
SELECT MIN(FechaTransaccion) AS fecha_mas_antigua, MAX(FechaTransaccion) AS fecha_mas_reciente
FROM WWImportersTransactional.movimientosCopia;


In [36]:
dataframes[3]

,fecha_mas_antigua,fecha_mas_reciente
0,2013-12-31 07:00:00.0000000,"Sep 30,2015"


Esta consulta determina las fechas de la transacción más antigua y más reciente en la tabla. Ayuda a verificar el periodo temporal que cubren los datos y a asegurarse de que todas las transacciones están dentro del rango esperado.

### Consulta SQL:
```sql
SELECT COUNT(DISTINCT ClienteID) AS clientes_unicos
FROM WWImportersTransactional.movimientosCopia;


In [37]:
dataframes[6]

,clientes_unicos
0,664


Esta consulta cuenta el número de clientes únicos en la tabla movimientosCopia. Es importante para evaluar la diversidad de la base de clientes y para cualquier análisis relacionado con el comportamiento de los clientes.

### Consulta SQL:
```sql
SELECT COUNT(*) AS registros_sin_clienteid
FROM WWImportersTransactional.movimientosCopia
WHERE ClienteID IS NULL OR ClienteID = '';


In [38]:
dataframes[12]

,registros_sin_clienteid
0,7156


Esta consulta cuenta el número de registros que no tienen un ClienteID asociado. Identificar estos registros es crucial para evaluar problemas de calidad de datos y para planificar acciones de limpieza o depuración de datos.

### Conclusión

Después de analizar los datos de la tabla `movimientosCopia` de la base de datos `WWImportersTransactional`, se obtuvieron los siguientes hallazgos:

1. **Estructura de los Datos:**
   - La tabla incluye columnas clave como cantidad de productos, identificadores de clientes y proveedores, fechas de transacción, y tipos de transacciones. Esto proporciona una base sólida para analizar el comportamiento del inventario.

2. **Volumen y Temporalidad:**
   - La tabla cubre un período significativo, lo que permite realizar análisis históricos detallados.

3. **Diversidad de Clientes:**
   - La alta diversidad en las transacciones de productos refleja una amplia base de clientes.

4. **Calidad de los Datos:**
   - Se detectaron problemas de calidad de datos que deben ser abordados para mejorar la precisión de los análisis.

### Recomendaciones

1. **Mejora de la Calidad:**
   - Implementar procesos de limpieza de datos para manejar registros incompletos y asegurar la integridad de los datos.

2. **Análisis Detallado:**
   - Realizar análisis segmentados por fechas, productos y clientes prioritarios para identificar patrones y tendencias.

3. **Monitoreo Continuo:**
   - Establecer mecanismos de monitoreo para rastrear el movimiento de inventario y las transacciones de productos, permitiendo identificar anomalías o tendencias emergentes.

Estas recomendaciones buscan mejorar la precisión y utilidad de los análisis, apoyando la toma de decisiones informadas alineadas con los objetivos estratégicos de WWImporters.


# Lo que sigue son el resto de consultas que realice en la base de datos. Por favor ignorar...

1. SELECT * FROM WWImportersTransactional.movimientosCopia LIMIT 10;

2. SELECT COUNT(*) AS total_registros FROM WWImportersTransactional.movimientosCopia;

3. SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'movimientosCopia' AND TABLE_SCHEMA = 'WWImportersTransactional';

4. SELECT MIN(FechaTransaccion) AS fecha_mas_antigua, MAX(FechaTransaccion) AS fecha_mas_reciente
FROM WWImportersTransactional.movimientosCopia;

5. SELECT MAX(Cantidad) AS cantidad_maxima
FROM WWImportersTransactional.movimientosCopia;

6. SELECT COUNT(*) AS movimientos_desde_2013
FROM WWImportersTransactional.movimientosCopia
WHERE FechaTransaccion >= '2013-01-01';

7. SELECT COUNT(DISTINCT ClienteID) AS clientes_unicos
FROM WWImportersTransactional.movimientosCopia;

8. SELECT COUNT(DISTINCT ProveedorID) AS proveedores_unicos
FROM WWImportersTransactional.movimientosCopia;

9. SELECT SUM(Cantidad) AS cantidad_total_productos
FROM WWImportersTransactional.movimientosCopia;

10. SELECT AVG(Cantidad) AS promedio_cantidad
FROM WWImportersTransactional.movimientosCopia;

11. SELECT strftime('%Y', FechaTransaccion) AS anio, COUNT(*) AS total_movimientos
FROM WWImportersTransactional.movimientosCopia
GROUP BY anio
ORDER BY anio;

12. SELECT *
FROM WWImportersTransactional.movimientosCopia
WHERE FechaTransaccion IS NULL OR FechaTransaccion = '';

13. SELECT COUNT(*) AS registros_sin_clienteid
    FROM WWImportersTransactional.movimientosCopia
    WHERE ClienteID IS NULL OR ClienteID = '';

14. SELECT COUNT(*) AS registros_sin_proveedorid
    FROM WWImportersTransactional.movimientosCopia
    WHERE ProveedorID IS NULL OR ProveedorID = '';

15. SELECT ClienteID, COUNT(*) AS total_movimientos_cliente
    FROM WWImportersTransactional.movimientosCopia
    GROUP BY ClienteID
    ORDER BY total_movimientos_cliente DESC
    LIMIT 10;

16. SELECT ProveedorID, COUNT(*) AS total_movimientos_proveedor
    FROM WWImportersTransactional.movimientosCopia
    GROUP BY ProveedorID
    ORDER BY total_movimientos_proveedor DESC
    LIMIT 10;

17. SELECT strftime('%Y-%m', FechaTransaccion) AS mes_anio, SUM(Cantidad) AS total_cantidad
    FROM WWImportersTransactional.movimientosCopia
    GROUP BY mes_anio
    ORDER BY mes_anio;

18. SELECT ProductoID, SUM(Cantidad) AS total_cantidad_producto
    FROM WWImportersTransactional.movimientosCopia
    GROUP BY ProductoID
    ORDER BY total_cantidad_producto DESC
    LIMIT 10;

19. SELECT COUNT(*) AS registros_sin_productoID
    FROM WWImportersTransactional.movimientosCopia
    WHERE ProductoID IS NULL OR ProductoID = '';

20. SELECT COUNT(*) AS registros_con_cantidad_negativa
    FROM WWImportersTransactional.movimientosCopia
    WHERE Cantidad < 0;

21. SELECT SUM(CASE WHEN Cantidad < 0 THEN Cantidad ELSE 0 END) AS total_cantidad_negativa,
           SUM(CASE WHEN Cantidad >= 0 THEN Cantidad ELSE 0 END) AS total_cantidad_positiva
    FROM WWImportersTransactional.movimientosCopia;

22. SELECT TipoMovimiento, COUNT(*) AS total_movimientos_tipo
    FROM WWImportersTransactional.movimientosCopia
    GROUP BY TipoMovimiento
    ORDER BY total_movimientos_tipo DESC;

23. SELECT COUNT(*) AS registros_con_fecha_futura
    FROM WWImportersTransactional.movimientosCopia
    WHERE FechaTransaccion > CURRENT_DATE;

24. SELECT COUNT(*) AS registros_sin_cantidad
    FROM WWImportersTransactional.movimientosCopia
    WHERE Cantidad IS NULL OR Cantidad = '';



In [2]:
dataframes[0]

,TransaccionProductoID,ProductoID,TipoTransaccionID,ClienteID,InvoiceID,ProveedorID,OrdenDeCompraID,FechaTransaccion,Cantidad
0,118903,217,10,476,24904,NaN,NaN,"Apr 25,2014",-40
1,286890,135,10,33,60117,NaN,NaN,"Dec 10,2015",-7
2,285233,111,10,180,59768,NaN,NaN,"Dec 04,2015",-2
3,290145,213,10,33,60795,NaN,NaN,"Dec 23,2015",-3
4,247492,90,10,55,51851,NaN,NaN,"Jul 27,2015",-24
5,157714,212,10,146,33043,NaN,NaN,"Sep 15,2014",-20
6,250024,218,10,585,52377,NaN,NaN,"Aug 04,2015",-60
7,200801,120,10,30,42049,NaN,NaN,"Feb 23,2015",-3
8,221525,129,10,596,46388,NaN,NaN,"May 01,2015",-7
9,295331,197,10,1037,61886,NaN,NaN,"Jan 08,2016",-216


In [6]:
dataframes[1]

,total_registros
0,204292


In [7]:
dataframes[2]

,COLUMN_NAME,DATA_TYPE
0,Cantidad,double
1,ClienteID,double
2,FechaTransaccion,text
3,InvoiceID,double
4,OrdenDeCompraID,text
5,ProductoID,int
6,ProveedorID,text
7,TipoTransaccionID,int
8,TransaccionProductoID,int


In [8]:
dataframes[3]

,fecha_mas_antigua,fecha_mas_reciente
0,2013-12-31 07:00:00.0000000,"Sep 30,2015"


In [9]:
dataframes[4]

,cantidad_maxima
0,67368


In [10]:
dataframes[5]

,movimientos_desde_2013
0,204292


In [11]:
dataframes[6]

,clientes_unicos
0,664


In [12]:
dataframes[7]

,proveedores_unicos
0,4


In [13]:
dataframes[8]

,cantidad_total_productos
0,146988046


In [14]:
dataframes[9]

,promedio_cantidad
0,719.499765


In [15]:
dataframes[10]

""


In [16]:
dataframes[11]

,TransaccionProductoID,ProductoID,TipoTransaccionID,ClienteID,InvoiceID,ProveedorID,OrdenDeCompraID,FechaTransaccion,Cantidad


In [17]:
dataframes[12]

,registros_sin_clienteid
0,7156


In [18]:
dataframes[13]

,registros_sin_proveedorid
0,197182


In [19]:
dataframes[14]

,ClienteID,total_movimientos_cliente
0,0,7156
1,980,409
2,810,401
3,954,395
4,804,395
5,149,395
6,185,391
7,953,389
8,438,385
9,569,385


In [20]:
dataframes[15]

,ProveedorID,total_movimientos_proveedor
0,NaN,197182
1,4.0,4832
2,7.0,2267
3,1.0,11


In [21]:
dataframes[16]

""


In [22]:
dataframes[17]

,ProductoID,total_cantidad_producto
0,193,28446480
1,78,20440692
2,98,20232201
3,86,20027712
4,77,18888300
5,204,13766002
6,184,13410750
7,95,9881136
8,80,9310620
9,114,-4389


In [23]:
dataframes[18]

,registros_sin_productoID
0,0


In [24]:
dataframes[19]

,registros_con_cantidad_negativa
0,197158


In [25]:
dataframes[20]

,total_cantidad_negativa,total_cantidad_positiva
0,-7714505,154702551


In [26]:
dataframes[21]

""


In [27]:
dataframes[22]

,registros_con_fecha_futura
0,0


In [28]:
dataframes[23]

,registros_sin_cantidad
0,5
